 # Ramp Technical Assessment

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import curve_fit

1) Weighted Retention curve for last 180 days

In [ ]:
a_series = a.iloc[819:999,:12]    #excluded last day as it has no retention rate yet

lst = []

for i in range(8):
  i += 4
  weighted = (a_series.iloc[:,i] * a_series["DNU"]).sum(skipna = True) / a_series["DNU"].sum(skipna = True)
  # need to exclude DNU values from denimonator when the Dx value is Null
  lst.append(weighted)

cols = a.iloc[1:2,4:12].columns

col = []

for i in cols:
    col_num = i.replace('D','')
    col.append(int(col_num))


a_sumdf = pd.DataFrame({'Dx': col , 'Weighted': lst})
a_sumdf.set_index('Dx', inplace=True)

tickvalues = a_sumdf.index
plt.xticks(ticks = tickvalues)
plt.plot(a_sumdf)
plt.show()